In [499]:
#Imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import numpy as np


**Process Data on Environmental Taxes**

In [500]:
#Set working directory
os.chdir('/Users/khangphamgia/Downloads')
#Load data
df_tax = pd.read_csv("/Users/khangphamgia/Downloads/Environmental_Taxes.csv")
#Drop multiple unncessary columns
df_tax = df_tax.drop(columns = ["ObjectId", "ISO2", "ISO3", "Source", "CTS Code", "CTS Name", "CTS Full Descriptor", "1995", "1996", "1997", "1998", "1999","2000", "2020", "2021"])
df_tax.head()

,Country,Indicator,Unit,2001,2002,2003,2004,2005,2006,2007,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Albania,Environmental Taxes,Domestic Currency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.399314e+10,4.781379e+10,4.754858e+10,5.114559e+10,5.341565e+10
1,Albania,Environmental Taxes,Percent of GDP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.067206e+00,3.247163e+00,3.066373e+00,3.124865e+00,3.157133e+00
2,Albania,Taxes on Energy (including fuel for transport),Domestic Currency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.774111e+10,4.094562e+10,4.040004e+10,4.352182e+10,4.516530e+10
3,Albania,Taxes on Energy (including fuel for transport),Percent of GDP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.631314e+00,2.780726e+00,2.605369e+00,2.659072e+00,2.669496e+00
4,Albania,Taxes on Pollution,Domestic Currency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.782069e+09,1.879970e+09,1.941324e+09,2.226251e+09,2.625011e+09


In [501]:
# Select Environmental Taxes
df_tax = df_tax[df_tax['Indicator'] == 'Environmental Taxes']
# Search for unique countries
df_tax["Country"].unique()

array(['Albania', 'Andorra, Principality of', 'Antigua and Barbuda',
       'Argentina', 'Armenia, Rep. of', 'Australia', 'Austria',
       'Bahamas, The', 'Bangladesh', 'Belgium', 'Belize', 'Bhutan',
       'Bolivia', 'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Chad', 'Chile',
       'China, P.R.: Mainland', 'Colombia', 'Congo, Dem. Rep. of the',
       'Congo, Rep. of', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire",
       'Croatia, Rep. of', 'Cyprus', 'Czech Rep.', 'Denmark',
       'Dominican Rep.', 'Ecuador', 'Egypt, Arab Rep. of', 'El Salvador',
       'Equatorial Guinea, Rep. of', 'Estonia, Rep. of',
       'Eswatini, Kingdom of', 'Fiji, Rep. of', 'Finland', 'France',
       'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guyana',
       'Honduras', 'Hungary', 'Iceland', 'Ireland', 'Israel', 'Italy',
       'Jamaica', 'Japan', 'Kazakhstan, Rep. of', 'Kenya',
       'Korea, Rep. of', 'Kosovo, Rep. of', 'Kyrgyz R

In [502]:
print(df_tax)

                             Country            Indicator               Unit  \
0                            Albania  Environmental Taxes  Domestic Currency   
1                            Albania  Environmental Taxes     Percent of GDP   
10          Andorra, Principality of  Environmental Taxes  Domestic Currency   
15               Antigua and Barbuda  Environmental Taxes  Domestic Currency   
16               Antigua and Barbuda  Environmental Taxes     Percent of GDP   
...                              ...                  ...                ...   
1210  Venezuela, Rep. Bolivariana de  Environmental Taxes     Percent of GDP   
1219                         Vietnam  Environmental Taxes  Domestic Currency   
1220                         Vietnam  Environmental Taxes     Percent of GDP   
1229              West Bank and Gaza  Environmental Taxes  Domestic Currency   
1230              West Bank and Gaza  Environmental Taxes     Percent of GDP   

             2001         2002         

In [503]:
# Step 1: Melt the dataframe to reshape year columns into rows
df_melted = df_tax.melt(id_vars=['Country', 'Indicator', 'Unit'], var_name='Year', value_name='Value')

# Step 2: Convert the Year column to numeric
df_melted['Year'] = pd.to_numeric(df_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
df_pivot = df_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Step 4: Rename columns for clarity
df_pivot.columns.name = None  # Remove multi-index column name
df_pivot.rename(columns={
    'Percent of GDP': 'Environmental Taxes (% of GDP)',
    'Domestic Currency': 'Environmental Taxes (Domestic Currency)'
}, inplace=True)

# Step 5: Create a full range of countries and years
all_countries = df_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2021))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
df_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
df_main = pd.merge(df_full, df_pivot, on=['Country', 'Year'], how='left')

# Step 6: Fill missing values with NaN or placeholders if necessary
df_main['Environmental Taxes (% of GDP)'] = df_main['Environmental Taxes (% of GDP)'].fillna(np.nan)
df_main['Environmental Taxes (Domestic Currency)'] = df_main['Environmental Taxes (Domestic Currency)'].fillna(np.nan)

# Final dataframe
print(df_main)

#Export to excel 
#df_main.to_excel("Environmental_Taxes_Cleaned.xlsx", index=False)



                 Country  Year  Environmental Taxes (Domestic Currency)  \
0                Albania  2000                                      NaN   
1                Albania  2001                                      NaN   
2                Albania  2002                                      NaN   
3                Albania  2003                                      NaN   
4                Albania  2004                                      NaN   
...                  ...   ...                                      ...   
2641  West Bank and Gaza  2016                             2.752039e+07   
2642  West Bank and Gaza  2017                             2.752039e+07   
2643  West Bank and Gaza  2018                             2.752039e+07   
2644  West Bank and Gaza  2019                             2.752039e+07   
2645  West Bank and Gaza  2020                                      NaN   

      Environmental Taxes (% of GDP)  
0                                NaN  
1                    

In [504]:
# Run summary statistics for each country
summary_stats = df_main.groupby('Country').agg({
    'Environmental Taxes (% of GDP)': ['mean', 'median', 'std', 'min', 'max', 'count'],
    'Environmental Taxes (Domestic Currency)': ['mean', 'median', 'std', 'min', 'max', 'count']
}).reset_index()
print(summary_stats)

                            Country Environmental Taxes (% of GDP)            \
                                                              mean    median   
0                           Albania                       3.132548  3.124865   
1          Andorra, Principality of                            NaN       NaN   
2               Antigua and Barbuda                       0.537895  0.630000   
3                         Argentina                       1.234211  1.220000   
4                  Armenia, Rep. of                       1.034888  1.049627   
..                              ...                            ...       ...   
121                   United States                       0.791067  0.752718   
122                         Uruguay                       1.674216  1.670845   
123  Venezuela, Rep. Bolivariana de                       2.177000  1.640000   
124                         Vietnam                       0.469412  0.430000   
125              West Bank and Gaza     

In [505]:
#Create a column for value count of each country for 2 variables in the original dataframe
df_main['count'] = df_main.groupby('Country')['Environmental Taxes (% of GDP)'].transform('count')
#Drop countries with less than 10 observations
df_main = df_main[df_main['count'] >= 10]
#Drop the count column
df_main = df_main.drop(columns = ['count'])
#Print the new dataframe
df_main["Country"].unique()


array(['Antigua and Barbuda', 'Argentina', 'Australia', 'Austria',
       'Bahamas, The', 'Bangladesh', 'Belgium', 'Belize', 'Bolivia',
       'Bulgaria', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Chad',
       'China, P.R.: Mainland', 'Congo, Dem. Rep. of the',
       'Congo, Rep. of', 'Costa Rica', 'Croatia, Rep. of', 'Cyprus',
       'Czech Rep.', "Côte d'Ivoire", 'Denmark', 'Ecuador',
       'Egypt, Arab Rep. of', 'Estonia, Rep. of', 'Eswatini, Kingdom of',
       'Fiji, Rep. of', 'Finland', 'France', 'Germany', 'Ghana', 'Greece',
       'Guyana', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Jamaica',
       'Japan', 'Kenya', 'Korea, Rep. of', 'Kyrgyz Rep.', 'Latvia',
       'Lithuania', 'Luxembourg', 'Malaysia', 'Maldives', 'Mali', 'Malta',
       'Mauritania, Islamic Rep. of', 'Mauritius', 'Mexico', 'Mongolia',
       'Morocco', 'Namibia', 'Netherlands, The', 'Nicaragua', 'Niger',
       'Nigeria', 'Norway', 'Panama', 'Papua New Guinea', 'Paraguay',
       'Peru', 'Philippines', 'Pola

In [506]:
import pandas as pd

# List of countries
countries = [
    'Antigua and Barbuda', 'Argentina', 'Australia', 'Austria',
    'Bahamas, The', 'Bangladesh', 'Belgium', 'Belize', 'Bolivia',
    'Bulgaria', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Chad',
    'China, P.R.: Mainland', 'Congo, Dem. Rep. of the',
    'Congo, Rep. of', 'Costa Rica', 'Croatia, Rep. of', 'Cyprus',
    'Czech Rep.', "Côte d'Ivoire", 'Denmark', 'Ecuador',
    'Egypt, Arab Rep. of', 'Estonia, Rep. of', 'Eswatini, Kingdom of',
    'Fiji, Rep. of', 'Finland', 'France', 'Germany', 'Ghana', 'Greece',
    'Guyana', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Jamaica',
    'Japan', 'Kenya', 'Korea, Rep. of', 'Kyrgyz Rep.', 'Latvia',
    'Lithuania', 'Luxembourg', 'Malaysia', 'Maldives', 'Mali', 'Malta',
    'Mauritania, Islamic Rep. of', 'Mauritius', 'Mexico', 'Mongolia',
    'Morocco', 'Namibia', 'Netherlands, The', 'Nicaragua', 'Niger',
    'Nigeria', 'Norway', 'Panama', 'Papua New Guinea', 'Paraguay',
    'Peru', 'Philippines', 'Poland, Rep. of', 'Portugal', 'Romania',
    'Rwanda', 'Serbia, Rep. of', 'Seychelles', 'Sierra Leone',
    'Slovak Rep.', 'Slovenia, Rep. of', 'Solomon Islands',
    'South Africa', 'St. Lucia', 'Sweden', 'Switzerland', 'Togo',
    'Trinidad and Tobago', 'Tunisia', 'Türkiye, Rep. of', 'Uganda',
    'United Kingdom', 'United States',
    'Venezuela, Rep. Bolivariana de', 'Vietnam'
]

# Country-region mapping
region_mapping = {
    'North America': ['Antigua and Barbuda', 'Bahamas, The', 'Belize', 'Canada', 'Costa Rica', 'Jamaica', 
                      'Mexico', 'Panama', 'St. Lucia', 'Trinidad and Tobago', 'United States'],
    'South America': ['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 
                      'Peru', 'Suriname', 'Uruguay', 'Venezuela, Rep. Bolivariana de'],
    'Europe': ['Austria', 'Belgium', 'Bulgaria', 'Croatia, Rep. of', 'Cyprus', 'Czech Rep.', 'Denmark', 
               'Estonia, Rep. of', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 
               'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands, The', 'Norway', 'Poland, Rep. of', 
               'Portugal', 'Romania', 'Serbia, Rep. of', 'Slovak Rep.', 'Slovenia, Rep. of', 'Spain', 'Sweden', 
               'Switzerland', 'United Kingdom'],
    'Asia': ['Bangladesh', 'Cambodia', 'China, P.R.: Mainland', 'India', 'Japan', 'Korea, Rep. of', 'Kyrgyz Rep.', 
             'Malaysia', 'Maldives', 'Mongolia', 'Philippines', 'Singapore', 'Thailand', 'Vietnam'],
    'Africa': ['Cameroon', 'Cabo Verde', 'Chad', 'Congo, Dem. Rep. of the', 'Congo, Rep. of', 
               'Egypt, Arab Rep. of', 'Eswatini, Kingdom of', 'Ghana', 'Kenya', 'Mali', 'Mauritania, Islamic Rep. of', 
               'Mauritius', 'Morocco', 'Namibia', 'Nigeria', 'Rwanda', 'Seychelles', 'Sierra Leone', 'South Africa', 
               'Togo', 'Tunisia', 'Uganda'],
    'Oceania': ['Australia', 'Fiji, Rep. of', 'Papua New Guinea', 'Solomon Islands'],
}

df_main['Region'] = df_main['Country'].map({country: region for region, countries in region_mapping.items() for country in countries})
df_main.head()





,Country,Year,Environmental Taxes (Domestic Currency),Environmental Taxes (% of GDP),Region
42,Antigua and Barbuda,2000,NaN,NaN,North America
43,Antigua and Barbuda,2001,19240000.0,0.89,North America
44,Antigua and Barbuda,2002,19340000.0,0.88,North America
45,Antigua and Barbuda,2003,18840000.0,0.82,North America
46,Antigua and Barbuda,2004,20040000.0,0.81,North America


**Process Data on Renewable Energy Share in Total Final Energy Consumption**

In [507]:
# Load the data 
renewable_energy = pd.read_excel("/Users/khangphamgia/Downloads/Renewable Energy Share in Total Final Energy Consumption (%).xlsx") 
# Rename the column 'Economy' to 'Country' to match the existing dataframe
renewable_energy.rename(columns={'Economy': 'Country'}, inplace=True)
renewable_energy.head()


,Indicator,Country,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Renewable Energy Share in Total Final Energy C...,Central and West Asia,NaN,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,Renewable Energy Share in Total Final Energy C...,Afghanistan,percent,44.99,45.6,37.83,36.66,44.24,33.88,31.89,...,19.06,17.69,20.18,19.48,18.28,18.87,18.21,19.98,...,...
2,Renewable Energy Share in Total Final Energy C...,Armenia,percent,7.16,5.42,8.9,9.81,8.71,6.49,7.65,...,7.08,10.74,13.15,12.56,11.13,10.34,8.38,9.13,...,...
3,Renewable Energy Share in Total Final Energy C...,Azerbaijan,percent,2.07,1.75,2.35,2.94,3.07,3.36,2.86,...,2.12,2.33,1.93,1.9,1.92,1.69,1.29,1.32,...,...
4,Renewable Energy Share in Total Final Energy C...,Georgia,percent,47.25,53.15,56.76,55.33,54.53,41.48,37.22,...,30.69,27.54,27.18,27.52,27.37,24.69,23.45,25.18,...,...


In [508]:
# Transform the data
#Melt the dataframe to reshape year columns into rows
renewable_energy_melted = renewable_energy.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
renewable_energy_melted['Year'] = pd.to_numeric(renewable_energy_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
renewable_energy_pivot = renewable_energy_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit of Measure'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
renewable_energy_pivot.columns.name = None  # Remove multi-index column name
renewable_energy_pivot.rename(columns={
    'percent': 'Renewable Energy Share in Total Final Energy Consumption (%)'
}, inplace=True)

# Create a full range of countries and years
all_countries = renewable_energy_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2022))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
renewable_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
renewable_main = pd.merge(renewable_full, renewable_energy_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
renewable_main['Renewable Energy Share in Total Final Energy Consumption (%)'] = renewable_main['Renewable Energy Share in Total Final Energy Consumption (%)'].fillna(np.nan)

print(renewable_main)

          Country  Year  \
0     Afghanistan  2000   
1     Afghanistan  2001   
2     Afghanistan  2002   
3     Afghanistan  2003   
4     Afghanistan  2004   
...           ...   ...   
1073     Viet Nam  2017   
1074     Viet Nam  2018   
1075     Viet Nam  2019   
1076     Viet Nam  2020   
1077     Viet Nam  2021   

     Renewable Energy Share in Total Final Energy Consumption (%)  
0                                                 44.99            
1                                                  45.6            
2                                                 37.83            
3                                                 36.66            
4                                                 44.24            
...                                                 ...            
1073                                              28.32            
1074                                              24.34            
1075                                              20.43        

**Process Data on Household' Final Consumption Expenditure**

In [509]:
# Load the data
household_cons = pd.read_excel("/Users/khangphamgia/Downloads/Household Consumption Expenditure (% of GDP).xlsx")
# Rename the column 'Economy' to 'Country' to match the existing dataframe
household_cons.rename(columns={'Economy': 'Country'}, inplace=True)

In [510]:
# Transform the data
household_cons_melted = household_cons.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
household_cons_melted['Year'] = pd.to_numeric(household_cons_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
household_cons_pivot = household_cons_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit of Measure'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
household_cons_pivot.columns.name = None  # Remove multi-index column name
household_cons_pivot.rename(columns={
    'percent of GDP': 'Household Consumption Expenditure (% of GDP)'
}, inplace=True)

# Create a full range of countries and years
all_countries = household_cons_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
household_cons_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
household_cons_main = pd.merge(household_cons_full, household_cons_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
household_cons_main['Household Consumption Expenditure (% of GDP)'] = household_cons_main['Household Consumption Expenditure (% of GDP)'].fillna(np.nan)

print(household_cons_main)

          Country  Year Household Consumption Expenditure (% of GDP)
0     Afghanistan  2000                                          ...
1     Afghanistan  2001                                          ...
2     Afghanistan  2002                                   111.175303
3     Afghanistan  2003                                   126.551644
4     Afghanistan  2004                                   120.511733
...           ...   ...                                          ...
1171     Viet Nam  2019                                     56.82807
1172     Viet Nam  2020                                    55.967075
1173     Viet Nam  2021                                        55.52
1174     Viet Nam  2022                                        54.79
1175     Viet Nam  2023                                    54.584566

[1176 rows x 3 columns]


**Process Data on Electricity Production**

In [511]:
elec_combust = pd.read_excel("/Users/khangphamgia/Downloads/Sources of Electricity, Combustible Fuels (% of total).xlsx")
elec_hydro = pd.read_excel("/Users/khangphamgia/Downloads/Sources of Electricity, Hydropower (% of total).xlsx")
elec_solar = pd.read_excel("/Users/khangphamgia/Downloads/Sources of Electricity, Solar (% of total).xlsx")
elec_other = pd.read_excel("/Users/khangphamgia/Downloads/Sources of Electricity, Others (% of total).xlsx")
total_elec_production = pd.read_excel("/Users/khangphamgia/Downloads/Total Electricity Production (kWh billion).xlsx")

In [512]:
elec_combust.head()

,Indicator,Economy,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"Sources of Electricity, Combustible Fuels (% o...",Central and West Asia,percent,69.688789,71.153958,69.200383,67.471898,68.260557,67.576221,68.847185,...,72.65956,71.974049,71.898018,72.822673,72.462003,70.721918,72.116388,71.533905,...,...
1,"Sources of Electricity, Combustible Fuels (% o...",Afghanistan,percent,25.010906,27.18854,20.936029,29.750271,29.109159,25.93819,29.24425,...,11.725664,11.145141,11.098357,12.412326,15.370718,11.239003,9.06571,9.226402,9.193777,...
2,"Sources of Electricity, Combustible Fuels (% o...",Armenia,percent,45.182947,48.563969,28.610255,27.649518,26.749585,28.937787,24.844302,...,42.43871,35.914861,35.283664,36.981457,43.3258,39.675347,40.383684,42.946161,...,...
3,"Sources of Electricity, Combustible Fuels (% o...",Azerbaijan,percent,91.796353,93.141441,89.198439,88.400827,87.329255,86.844176,89.740456,...,94.722582,93.32999,91.915168,92.575521,92.50868,93.42458,95.307101,94.890777,...,...
4,"Sources of Electricity, Combustible Fuels (% o...",Georgia,percent,21.06681,20.109557,6.752101,8.854749,12.637204,14.187423,29.069614,...,19.631665,21.95892,19.315344,19.364854,17.40859,23.955873,25.276439,18.818357,23.779184,...


In [ ]:
elec_hydro.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
elec_hydro_melted = elec_hydro.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
elec_hydro_melted['Year'] = pd.to_numeric(elec_hydro_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
elec_hydro_pivot = elec_hydro_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit of Measure'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
elec_hydro_pivot.columns.name = None  # Remove multi-index column name
elec_hydro_pivot.rename(columns={
    'percent': 'Sources of Electricity, Hydropower (% of Total)'
}, inplace=True)

# Create a full range of countries and years
all_countries = elec_hydro_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
elec_hydro_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
elec_hydro_main = pd.merge(elec_hydro_full,elec_hydro_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
elec_hydro_main['Sources of Electricity, Hydropower (% of Total)'] = elec_hydro_main['Sources of Electricity, Hydropower (% of Total)'].fillna(np.nan)

print(elec_hydro_main)


In [ ]:
elec_combust.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
elec_combust_melted = elec_combust.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
elec_combust_melted['Year'] = pd.to_numeric(elec_combust_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
elec_combust_pivot = elec_combust_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit of Measure'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
elec_combust_pivot.columns.name = None  # Remove multi-index column name
elec_combust_pivot.rename(columns={
    'percent': 'Sources of Electricity, Combustible Fuels (% of Total)'
}, inplace=True)

# Create a full range of countries and years
all_countries = elec_combust_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
elec_combust_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
elec_combust_main = pd.merge(elec_combust_full,elec_combust_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
elec_combust_main['Sources of Electricity, Combustible Fuels (% of Total)'] = elec_combust_main['Sources of Electricity, Combustible Fuels (% of Total)'].fillna(np.nan)

print(elec_combust_main)

In [515]:
elec_solar.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
elec_solar_melted = elec_solar.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
elec_solar_melted['Year'] = pd.to_numeric(elec_solar_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
elec_solar_pivot = elec_solar_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit of Measure'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
elec_solar_pivot.columns.name = None  # Remove multi-index column name
elec_solar_pivot.rename(columns={
    'percent': 'Sources of Electricity, Solar (% of Total)'
}, inplace=True)

# Create a full range of countries and years
all_countries = elec_solar_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
elec_solar_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
elec_solar_main = pd.merge(elec_solar_full,elec_solar_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
elec_solar_main['Sources of Electricity, Solar (% of Total)'] = elec_solar_main['Sources of Electricity, Solar (% of Total)'].fillna(np.nan)

print(elec_solar_main)

          Country  Year Sources of Electricity, Solar (% of Total)
0     Afghanistan  2000                                        ...
1     Afghanistan  2001                                        ...
2     Afghanistan  2002                                        ...
3     Afghanistan  2003                                        ...
4     Afghanistan  2004                                        ...
...           ...   ...                                        ...
1171     Viet Nam  2019                                   2.146585
1172     Viet Nam  2020                                   3.921964
1173     Viet Nam  2021                                  10.962988
1174     Viet Nam  2022                                        ...
1175     Viet Nam  2023                                        ...

[1176 rows x 3 columns]


In [516]:
elec_other.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
elec_other_melted = elec_other.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
elec_other_melted['Year'] = pd.to_numeric(elec_other_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
elec_other_pivot = elec_other_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit of Measure'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
elec_other_pivot.columns.name = None  # Remove multi-index column name
elec_other_pivot.rename(columns={
    'percent': 'Sources of Electricity, Others (% of Total)'
}, inplace=True)

# Create a full range of countries and years
all_countries = elec_other_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
elec_other_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
elec_other_main = pd.merge(elec_other_full,elec_other_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
elec_other_main['Sources of Electricity, Others (% of Total)'] = elec_other_main['Sources of Electricity, Others (% of Total)'].fillna(np.nan)

print(elec_other_main)

      Country  Year Sources of Electricity, Others (% of Total)
0     Armenia  2000                                   33.652232
1     Armenia  2001                                   34.586597
2     Armenia  2002                                    41.34807
3     Armenia  2003                                   36.320669
4     Armenia  2004                                   39.850746
..        ...   ...                                         ...
811  Viet Nam  2019                                    0.134135
812  Viet Nam  2020                                    0.405749
813  Viet Nam  2021                                    1.319929
814  Viet Nam  2022                                         ...
815  Viet Nam  2023                                         ...

[816 rows x 3 columns]


In [518]:
total_elec_production.head()

,Indicator,Economy,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Total Electricity Production (kWh billion),Central and West Asia,kilowatt-hour,237102700000,243261100000,254610311100,271844944400,282786944400,294432055600,307448944400,...,354875880000,356576625000,375123137000,399731163000,410947782000,410262199000,425336681000,438721764000,...,...
1,Total Electricity Production (kWh billion),Afghanistan,kilowatt-hour,687700000,691100000,722200000,921000000,797000000,906000000,913000000,...,1356000000,1338700000,1393900000,1420040000,1300720000,1625500000,1370880000,1409000000,1414000000,...
2,Total Electricity Production (kWh billion),Armenia,kilowatt-hour,5958000000,5745000000,5519000000,5501000000,6030000000,6317000000,5941000000,...,7750000000,7799000000,7315000000,7765243000,7791196000,7679643000,7838683000,7879014000,...,...
3,Total Electricity Production (kWh billion),Azerbaijan,kilowatt-hour,18699000000,18969000000,18701000000,21286000000,21743000000,22872000000,24543000000,...,24728000000,24688420000,24952900000,24320900000,25229200000,26072903000,25839103000,27888801000,...,...
4,Total Electricity Production (kWh billion),Georgia,kilowatt-hour,7424000000,6937000000,7257000000,7160000000,6924000000,7267000000,7599000000,...,10371000000,10832500000,11573700000,11531200000,12148600000,11856800000,11159800000,12645100000,14246620000,...


In [520]:
total_elec_production.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
total_elec_production_melted = total_elec_production.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
total_elec_production_melted['Year'] = pd.to_numeric(total_elec_production_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
total_elec_production_pivot = total_elec_production_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Indicator'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
total_elec_production_pivot.columns.name = None  # Remove multi-index column name
total_elec_production_pivot.rename(columns={
    'Total Electricity Production (kWh billion)': 'Total Electricity Production (kWh billion)'
}, inplace=True)

# Create a full range of countries and years
all_countries = total_elec_production_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
total_elec_production_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
total_elec_production_main = pd.merge(total_elec_production_full,total_elec_production_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
total_elec_production_main['Total Electricity Production (kWh billion)'] = total_elec_production_main['Total Electricity Production (kWh billion)'].fillna(np.nan)

print(total_elec_production_main)

          Country  Year Total Electricity Production (kWh billion)
0     Afghanistan  2000                                  687700000
1     Afghanistan  2001                                  691100000
2     Afghanistan  2002                                  722200000
3     Afghanistan  2003                                  921000000
4     Afghanistan  2004                                  797000000
...           ...   ...                                        ...
1315     Viet Nam  2019                               232602000000
1316     Viet Nam  2020                               240542724000
1317     Viet Nam  2021                               253498401000
1318     Viet Nam  2022                                        ...
1319     Viet Nam  2023                                        ...

[1320 rows x 3 columns]


In [527]:
#Merge all dataframes
df_electricity = pd.merge(elec_combust_main, elec_hydro_main, on=['Country', 'Year'], how='left')
df_electricity = pd.merge(df_electricity, elec_solar_main, on=['Country', 'Year'], how='left')
df_electricity = pd.merge(df_electricity, elec_other_main, on=['Country', 'Year'], how='left')
df_electricity = pd.merge(df_electricity, total_elec_production_main, on=['Country', 'Year'], how='left')
# Print country Vietnam
df_electricity[df_electricity['Country'] == 'Viet Nam']

,Country,Year,"Sources of Electricity, Combustible Fuels (% of Total)","Sources of Electricity, Hydropower (% of Total)","Sources of Electricity, Solar (% of Total)","Sources of Electricity, Others (% of Total)",Total Electricity Production (kWh billion)
1272,Viet Nam,2000,45.216671,54.783329,...,...,26561000000
1273,Viet Nam,2001,40.50575,59.49425,...,...,30608000000
1274,Viet Nam,2002,49.161918,50.838082,...,...,35796000000
1275,Viet Nam,2003,53.607819,46.392181,...,...,40925000000
1276,Viet Nam,2004,61.440412,38.559588,...,...,46209000000
1277,Viet Nam,2005,68.419189,31.580811,...,...,53656000000
1278,Viet Nam,2006,66.263865,33.736135,...,...,60493000000
1279,Viet Nam,2007,65.623508,34.376492,...,...,67008000000
1280,Viet Nam,2008,64.593438,35.405199,...,0.001362,73396000000
1281,Viet Nam,2009,63.94229,36.045687,...,0.012023,83175000000


**Process Economy Table Data**